In [1]:
import pickle
import gzip

import pandas as pd
import numpy as np
import scipy.sparse as sp

from tqdm import tqdm_notebook as tqdm
from sklearn.metrics import roc_auc_score

import competition_utils as u

In [2]:
X_train = sp.load_npz('tmp/X_train_sparse.npz')
X_val = sp.load_npz('tmp/X_val_sparse.npz')

y_train = np.load('tmp/y_train.npy', )
y_val = np.load('tmp/y_val.npy', )

y_train = y_train.astype(np.float32)
y_val = y_val.astype(np.float32)

In [3]:
import ftrl

In [6]:
model = ftrl.FtrlProximal(alpha=0.1, beta=1, l1=75, l2=25)

In [7]:
for i in tqdm(range(30)):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    auc = roc_auc_score(y_val, y_pred)
    print(i + 1, auc)

0.724328867473
0.728590625559
0.730495705164
0.731532878902
0.732231844931
0.73270145889
0.733032186926
0.733351507954
0.733541617152
0.733776918727
0.733888759805
0.733943977512
0.733985556024
0.734170677925
0.734128688444
0.734216945388
0.734240800076
0.734249430881
0.734360581606
0.734338162489
0.734446227374
0.734390811491
0.734452169826
0.734406550549
0.734359516496
0.73438159352
0.73444946328
0.734416310455
0.734439298542
0.734445272918



In [8]:
X = sp.vstack([X_train, X_val])
y = np.concatenate([y_train, y_val])

Full model

In [9]:
%%time

model_full = ftrl.FtrlProximal(alpha=0.1, beta=1, l1=75, l2=25)
model_full.fit(X, y, num_passes=22)

CPU times: user 17min 58s, sys: 1.32 s, total: 18min
Wall time: 2min 31s


In [10]:
shift = 1.1875
scale = 850100

def shifted_scaled_sigmoid(x, shift=0, scale=1):
    s = 1 / (1 + np.exp(-x + shift))
    return (s * scale).round(2)

In [13]:
it_test = u.read_grouped('data/criteo_test_release.txt.gz')

In [12]:
f_out = open('pred_ftrl2.txt', 'w')

for gid, group in tqdm(it_test, total=7087738):
    cols = []
    vals = []

    for line in group:
        cols.append(line.idx)
        vals.append(line.val)

    X_val = u.to_csr(cols, vals)

    pred = model_full.predict(X_val)
    pred = shifted_scaled_sigmoid(pred, shift, scale)

    pred_str = u.to_prediction_str(gid, pred)
    
    f_out.write(pred_str)
    f_out.write('\n')

f_out.flush()
f_out.close()

In [14]:
!gzip pred_ftrl2.txt

In [15]:
import crowdai
challenge = crowdai.Challenge("CriteoAdPlacementNIPS2017", 'd671d30799fa215f63a5cb5049983c79')

CrowdAI.Event.Authentication : Authenticating for challenge = CriteoAdPlacementNIPS2017
CrowdAI.Event.Connection.CONNECTED
CrowdAI.Event.Authentication.SUCCESS : Authentication Successful


In [16]:
scores = challenge.submit('pred_ftrl2.txt.gz')

CrowdAI.Event.Misc.FILE_UPLOAD : Preparing for file upload


  0% |          |[00:00<00:40]   2.45% /s] 

CrowdAI.Event.Misc.FILE_UPLOAD : Uploading file


100% |██████████|[00:08<00:00]  12.23% /s] 
  0% |          |[00:00<?]  ?% /s] 

CrowdAI.Event.Job.ENQUEUED : 7760cc63-c4fe-49dd-b994-e0ee3485de3b


...

7760cc63-c4fe-49dd-b994-e0ee3485de3b: 100% |█████████▉|[1:26:38<00:00]  52.22s/% ] 

CrowdAI.Event.Job.INFO : (7760cc63-c4fe-49dd-b994-e0ee3485de3b) Scores Computed Successfully !!
CrowdAI.Event.Job.INFO : (7760cc63-c4fe-49dd-b994-e0ee3485de3b) ImpWt_std: 0.0134043441367
CrowdAI.Event.Job.INFO : (7760cc63-c4fe-49dd-b994-e0ee3485de3b) Uploading scores to the leaderboard....
CrowdAI.Event.Job.INFO : (7760cc63-c4fe-49dd-b994-e0ee3485de3b) SNIPS_std: 0.000993291621552
CrowdAI.Event.Job.INFO : (7760cc63-c4fe-49dd-b994-e0ee3485de3b) SNIPS : 53.0539863777
CrowdAI.Event.Job.INFO : (7760cc63-c4fe-49dd-b994-e0ee3485de3b) ImpWt : 0.990946687511
CrowdAI.Event.Job.INFO : (7760cc63-c4fe-49dd-b994-e0ee3485de3b) IPS_std: 2.53459631776
CrowdAI.Event.Job.INFO : (7760cc63-c4fe-49dd-b994-e0ee3485de3b) IPS : 52.5736720603


7760cc63-c4fe-49dd-b994-e0ee3485de3b: 100% |██████████|[1:26:39<00:00]  257.11s/% ] 

CrowdAI.Event.Job.INFO : (7760cc63-c4fe-49dd-b994-e0ee3485de3b) Scores Submited Successfully !!! 
CrowdAI.Event.Job.COMPLETE : 7760cc63-c4fe-49dd-b994-e0ee3485de3b	   🍺 


7760cc63-c4fe-49dd-b994-e0ee3485de3b: 100% |██████████|[1:26:39<00:00]  257.11s/% ] 


In [17]:
scores

{'impwt': 0.9909466875114865,
 'impwt_std': 0.013404344136707162,
 'ips': 52.57367206026822,
 'ips_std': 2.5345963177595756,
 'max_instances': 7087738,
 'message': '',
 'snips': 53.05398637770695,
 'snips_std': 0.0009932916215519697}

In [154]:
scores

{'impwt': 0.983055672490548,
 'impwt_std': 0.015840261443422165,
 'ips': 46.11893916159966,
 'ips_std': 1.885006865357641,
 'max_instances': 7087738,
 'message': '',
 'snips': 46.91386302136728,
 'snips_std': 0.0007714217927742332}